# Bucketing

## Подготовка Данных

In [ ]:
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import col, sum, hash, expr
from pyspark.sql import functions as F
from pyspark.sql.dataframe import DataFrame

In [ ]:
spark = (
  SparkSession
    .builder
    .appName("Bucketing")
    .master("local[4]")
    .config("spark.sql.sources.bucketing.enabled", True)
    .config("spark.sql.autoBroadcastJoinThreshold", "-1")
    .config("spark.sql.warehouse.dir", "/tmp/spark-warehouse")
    .getOrCreate()
)

sc = spark.sparkContext

In [ ]:
! cd /tmp && rm -rf steam && rm -rf /tmp/spark-warehouse && unzip ~/notebooks/steam.zip && cd -

In [ ]:
recomendations_df = spark.read.parquet("/tmp/steam/recomendations.parquet")
users_df = spark.read.parquet("/tmp/steam/users.parquet")

In [ ]:
users_df.show(5)

In [ ]:
recomendations_df.repartition(4).write.mode("overwrite").parquet("/tmp/recomendations")
users_df.repartition(4).write.mode("overwrite").parquet("/tmp/users")

In [ ]:
recomendations_df = spark.read.parquet("/tmp/recomendations")
users_df = spark.read.parquet("/tmp/users")

In [ ]:
users_df.join(recomendations_df, "user_id").show(5)

In [ ]:
recomendations_df.printSchema()

In [ ]:
users_df.printSchema()

## Подготовка Bucketing

In [ ]:
print(f"Всего пользователей: {users_df.count()}\nВсего рекомендаций: {recomendations_df.count()}")

Проверить в каком бакете окажется значение можно при помощи функции [`pmod`](https://spark.apache.org/docs/3.3.2/sql-ref-functions-builtin.html#mathematical-functions):

In [ ]:
(
  users_df
    .limit(10)
    .withColumn("hash", hash(col("user_id")))
    .withColumn("bucket", expr("pmod(hash, 100)"))
    .select("user_id", "hash", "bucket")
    .show()
)

In [ ]:
def bucketing(df: DataFrame, table_name: str, column: str, buckets: int = 100) -> DataFrame:
    (
    df.write
      .mode("overwrite")
      .bucketBy(buckets, column)
      .sortBy(column)
      .saveAsTable(table_name)
    )
    return spark.table(table_name)

In [ ]:
recomendations = bucketing(recomendations_df, "recomendations", "user_id")
users = bucketing(users_df, "users", "user_id")

In [ ]:
spark.sql("DESCRIBE EXTENDED users").show(200, False)

In [ ]:
spark.sql("DESCRIBE EXTENDED recomendations").show(200, False)

## Использование Bucketing

### Исключение партиций

In [ ]:
users_df.where("user_id == 5693478").count()

In [ ]:
users.where("user_id == 5693478").count()

In [ ]:
users_df.where("reviews == 123").count()

In [ ]:
users.where("reviews == 123").count()

### Снижение shuffle операций в join

При объединении датафрейма с таблицей, по полю, по которому таблица разбита на бакеты, shuffle будет присутствовать только на стороне датафрейма:

In [ ]:
df = users_df.join(recomendations, "user_id")

In [ ]:
df.explain()

Такой тип объединения/join называется **one-side shuffle-free join**

In [ ]:
df.show(5)

### Join без shuffle

In [ ]:
df = users.join(recomendations, "user_id")

In [ ]:
df.explain()

In [ ]:
df.count()

Так же можно сразу исключить ненужные бакеты из анализа:

In [ ]:
df = (
  users
    .join(recomendations, "user_id")
    .where(col("user_id") == 5693478)
)

In [ ]:
df.explain()

In [ ]:
e = df.collect()

In [ ]:
e = users_df.join(recomendations_df, "user_id").where(col("user_id") == 5693478).collect()

In [ ]:
df = (
  users.hint("SHUFFLE_HASH")
    .join(recomendations, "user_id")
    .where(col("user_id") == 5693478)
)
df.explain()

In [ ]:
df = (
  users.hint("BROADCAST")
    .join(recomendations, "user_id")
    .where(col("user_id") == 5693478)
)
df.explain()

In [ ]:
e = df.collect()

### Сортировка

In [ ]:
def bucketing_single_partition(df: DataFrame, table_name: str, column: str, buckets: int = 100) -> DataFrame:
    (
    df.repartition(1)
      .write
      .mode("overwrite")
      .bucketBy(buckets, column)
      .sortBy(column)
      .saveAsTable(table_name)
    )
    return spark.table(table_name)

In [ ]:
recomendations_single_partition = bucketing_single_partition(
  recomendations_df,
  "recomendations_single_partition",
  "user_id"
)
users_single_partition = bucketing_single_partition(
  users_df,
  "users_single_partition",
  "user_id"
)

In [ ]:
spark.sql("DESC EXTENDED recomendations_single_partition").show(40, False)

In [ ]:
spark.sql("DESC EXTENDED users_single_partition").show(40, False)

In [ ]:
users.join(recomendations, "user_id").explain()

In [ ]:
users_single_partition.join(recomendations_single_partition, "user_id").explain()

In [ ]:
spark.conf.get("spark.sql.legacy.bucketedTableScan.outputOrdering")

In [ ]:
spark.conf.set("spark.sql.legacy.bucketedTableScan.outputOrdering", True)

In [ ]:
users_single_partition.join(recomendations_single_partition, "user_id").explain()

### Аггрегаты без shuffle

In [ ]:
users_df.groupby('user_id').count().explain()

In [ ]:
users.groupby('user_id').count().explain()

In [ ]:
df = (
  users_df
    .withColumn(
      'cnt',
      F.count('*')
        .over(Window().partitionBy('user_id'))
    )
  )
df.explain()

In [ ]:
df = (
  users
    .withColumn(
      'cnt',
      F.count('*')
        .over(Window().partitionBy('user_id'))
    )
  )
df.explain()

In [ ]:
df = (
  users_single_partition
    .withColumn(
      'cnt',
      F.count('*')
        .over(Window().partitionBy('user_id'))
    )
  )
df.explain()

## Особенности

### Неравное количество бакетов

In [ ]:
recomendations = bucketing(recomendations_df, "recomendations", "user_id", 50)

In [ ]:
spark.sql("DESC EXTENDED recomendations").where("col_name == 'Num Buckets'").show()

In [ ]:
spark.sql("DESC EXTENDED users").where("col_name == 'Num Buckets'").show()

In [ ]:
users.join(recomendations, "user_id").explain()

In [ ]:
(
users.repartition(50, "user_id")
  .join(recomendations, "user_id")
  .explain()
)

In [ ]:
spark.conf.get("spark.sql.bucketing.coalesceBucketsInJoin.enabled")

In [ ]:
spark.conf.set("spark.sql.bucketing.coalesceBucketsInJoin.enabled", True)

In [ ]:
# автоматическое склеивание партиций не работает при включенном AQE. Обновления в https://github.com/apache/spark/pull/40688
spark.conf.set("spark.sql.adaptive.enabled", False)

In [ ]:
users.join(recomendations, "user_id").explain()

In [ ]:
spark.conf.get("spark.sql.bucketing.coalesceBucketsInJoin.maxBucketRatio")

In [ ]:
spark.conf.set("spark.sql.bucketing.coalesceBucketsInJoin.maxBucketRatio", 10)

In [ ]:
users.join(recomendations, "user_id").explain(True)

In [ ]:
spark.sql("select /*+ SHUFFLE_HASH(u) */ * from users u join recomendations using (user_id)").explain()

### Загрузка данных

Spark хранит метаинформацию о том, что конкретная таблица имеет конкретное количество бакетов, поэтому обязательно необходимо создавать датафрейм из таблицы:

In [ ]:
(
  spark
    .table("users")
    .groupby('user_id')
    .count()
    .explain()
)

Если загрузить данные просто как parquet, то бакетинг исползоваться не будет:

In [ ]:
(
  spark
    .read.parquet('/tmp/spark-warehouse/users')
    .groupby('user_id')
    .count()
    .explain()
)

### Разные типы данных ключа

In [ ]:
users_df_long_key = (
  users_df
    .limit(1000)
    .withColumn(
      'user_id_long',
      col('user_id').cast("long")
    )
)
users_df_long_key.printSchema()

In [ ]:
users_long_key = bucketing(users_df_long_key, "users_long_key", "user_id_long")

In [ ]:
users_long_key.groupby('user_id_long').count().explain()

In [ ]:
(
  users_df_long_key
    .join(
      recomendations,
      users_df_long_key.user_id_long == recomendations.user_id
    )
    .explain()
)

In [ ]:
users_id_long = users.withColumn("user_id_long", col("user_id"))
(
  users_id_long
    .join(
      recomendations,
      users_id_long.user_id_long == recomendations.user_id
    )
    .explain()
)

### Пользовательские функции

In [ ]:
many_reviews = lambda reviews: reviews > 5
many_reviews_udf = F.udf(many_reviews)

In [ ]:
(
  users.withColumn(
      "many_reviews",
      many_reviews_udf(col("reviews"))
    )
    .join(recomendations, "user_id")
    .explain()
)

In [ ]:
(
  users
    .join(recomendations, "user_id")
    .withColumn(
      "many_reviews",
      many_reviews_udf(col("reviews"))
    )
    .explain()
)

### Динамическое партиционирование

In [ ]:
recomendations_part_by_year_df = recomendations_df \
  .withColumn(
    "year",
    F.date_format(col("date"), "y")
  )

In [ ]:
def bucketing_partition(df: DataFrame, table_name: str, column: str, partition: str, buckets: int = 100) -> DataFrame:
    (
    df.write
      .mode("overwrite")
      .bucketBy(buckets, column)
      .partitionBy(partition)
      .sortBy(column)
      .saveAsTable(table_name)
    )
    return spark.table(table_name)

In [ ]:
recomendations_part_by_year = bucketing_partition(recomendations_part_by_year_df, "recomendations_part_by_year", "user_id", "year")

In [ ]:
recomendations_part_by_year_df.where('year == 2012').explain()

In [ ]:
recomendations_part_by_year.where('year == 2012').explain(mode="formatted")

In [ ]:
(
recomendations_part_by_year
  .where('year == 2013')
  .limit(5)
  .withColumn('year', F.lit("2023"))
  .explain()
)

In [ ]:
new_data = (
  recomendations_part_by_year
    .where('year == 2013')
    .withColumn('date', F.add_months(col('date'), 10 * 12))
    .withColumn('year', F.lit("2023"))
    .limit(5)
  )

new_data.explain()

In [ ]:
new_data.write.insertInto("recomendations_part_by_year")

In [ ]:
recomendations_part_by_year.where("year = 2023").explain(mode="formatted")

## Конфигурация

- `spark.sql.sources.bucketing.enabled` — включить поддержку бакетов? По умолчанию True.
- `spark.sql.sources.bucketing.maxBuckets` — максимальное количество бакетов на таблицу. По умолчанию 100000 (100 тысяч).
- `spark.sql.sources.bucketing.autoBucketedScan.enabled` — Исключить использование бакетов из плана запроса, если без них эффективнее. По умолчанию True.
- `spark.sql.bucketing.coalesceBucketsInJoin.enabled` — если две таблицы при join имеют разное количество бакетов, слить (coalesce) несколько бакетов большей таблицы в один, чтобы уровнять число бакетов. Сработает только, если большее количество бакетов без остатка делится на меньшее (100 и 50, например). По умолчанию False
- `spark.sql.bucketing.coalesceBucketsInJoin.maxBucketRatio` — во сколько раз сколько количество бакетов может отличаться, чтобы выполнить слияние согласно настройке `spark.sql.bucketing.coalesceBucketsInJoin.enabled` (см. выше). По умолчанию, не больше чем в 4 раза.
- `spark.sql.legacy.bucketedTableScan.outputOrdering` — исключать сортировку из плана запроса, если бакет состоит из одного файла и все данные в нем отсортированы. По умолчанию False.